In [ ]:
import numpy as np
import cv2

def haar_transform(matrix):
    """
    Apply Haar wavelet transform on a 2D matrix.
    """
    size = len(matrix)
    transformed = np.zeros((size, size))
    while size > 1:
        half_size = size // 2
        for i in range(half_size):
            transformed[i][0:half_size] = (matrix[2*i][0:half_size] + matrix[2*i + 1][0:half_size]) / 2
            transformed[i][half_size:size] = (matrix[2*i][half_size:size] - matrix[2*i + 1][half_size:size]) / 2
            transformed[half_size + i][0:half_size] = (matrix[2*i][0:half_size] - matrix[2*i + 1][0:half_size]) / 2
            transformed[half_size + i][half_size:size] = (matrix[2*i][half_size:size] + matrix[2*i + 1][half_size:size]) / 2
        matrix = transformed[:half_size * 2, :half_size * 2]
        size = half_size
    return transformed

def inverse_haar_transform(transformed):
    """
    Apply inverse Haar wavelet transform.
    """
    size = len(transformed)
    matrix = np.copy(transformed)
    while size <= len(transformed):
        half_size = size // 2
        for i in range(half_size):
            matrix[2*i][0:half_size] = transformed[i][0:half_size] + transformed[half_size + i][0:half_size]
            matrix[2*i + 1][0:half_size] = transformed[i][0:half_size] - transformed[half_size + i][0:half_size]
            matrix[2*i][half_size:size] = transformed[i][half_size:size] + transformed[half_size + i][half_size:size]
            matrix[2*i + 1][half_size:size] = transformed[i][half_size:size] - transformed[half_size + i][half_size:size]
        size *= 2
    return matrix

def compress_image(img, threshold):
    """
    Compress image using Haar wavelet transform.
    """
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rows, cols = gray_img.shape
    compressed_img = np.zeros((rows, cols))

    for i in range(0, rows, 2):
        for j in range(0, cols, 2):
            block = gray_img[i:i + 2, j:j + 2]
            transformed_block = haar_transform(block)
            # Apply thresholding for compression
            for m in range(2):
                for n in range(2):
                    if abs(transformed_block[m][n]) < threshold:
                        transformed_block[m][n] = 0
            compressed_block = inverse_haar_transform(transformed_block)
            compressed_img[i:i + 2, j:j + 2] = compressed_block

    return compressed_img.astype(np.uint8)

# Load an image
image_path = 'path_to_your_image.jpg'
image = cv2.imread(image_path)

# Set compression threshold
compression_threshold = 20  # Adjust this value for desired compression level

# Compress the image
compressed_image = compress_image(image, compression_threshold)

# Display the original and compressed images
cv2.imshow('Original Image', image)
cv2.imshow('Compressed Image', compressed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
